LSTM Model
1. Make sure all packages are installed (WINDOWS)

*    pip install numpy
*   pip install pandas
*   pip install os-sys
*    pip install matplotlib
*   pip install seaborn
*   pip install scikit-learn
*   pip install keras
*   pip install tensorflow 

1. Make sure all packages are installed (MACS AND/OR ANACONDA ENVIRONMENT)
*   conda  install numpy
*   conda  install pandas
*   conda  install os-sys
*   conda  install matplotlib
*   conda  install seaborn
*   conda  install scikit-learn
*   conda  install keras
*   conda  install tensorflow 

Links:

Kaggle Data: https://www.kaggle.com/harnoor343/fall-detection-accelerometer-data 

Potential Issues:
 * can I reshape before putting in LSTM layer :(

Todo:
* test script and sensordata

In [ ]:
#DO NOT RUN IF YOU ARE NOT USING WINDOWS AND/OR NOT USING ANACONDA
%pip install numpy
%pip install pandas
%pip install os-sys
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install keras
%pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import sklearn.model_selection

#Yeah... that's all
run()


In [2]:
#Runs the entire machine learning model
def run():
    num = int(input("Choose a number from 1-3 to run the model with the correspdonding labels: \n 1. runfall, downSit, freeFall, runSit, walkFall, walkSit \n 2. runfall, downSit, freeFall, runSit, walkFall, walkSit, walkStand, walkWalk \n 3. runfall, downSit, freeFall, runSit, walkFall, walkSit, walkStand, walkWalk, runRun, runStand, standStand, standWalk \n"))
    df = data_collection(num)
    df = data_normalize(df)
    #data_plot(df)
    df = data_categorical(df)
    x_train, x_test, y_train, y_test = data_split(df)
    model = model_create(x_train)
    model = train_and_accurary_model(model, x_train, x_test, y_train, y_test)
    model_save(model)

    #debug purposes
    print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

The data is stored like this:
['SubjectID', 'Device','ActivityID','TrialNo','Acc','Gyr','Mag','Bar']
- The label (output) is the ActivityID
    - These labels are specified in ActivityID2Str.m


In [1]:
def data_collection(num = 1):
    #1: runfall, downSit, freeFall, runSit, walkFall, walkSit
    #2: ActivityID labels
    # create a new directory called 'normalized', ignore it if it already exists
    os.makedirs('./normalized', exist_ok=True)


    all_labels = {1:['runfall', 'downSit', 'freeFall', 'runSit', 'walkFall', 'walkSit'], 2: list(range(1,136))}
    labels = all_labels[num]
    for label in labels:
        #read all csv files in the directory "archive' + label, delimit by ';'
        #files = glob.glob('./archive/' + label + '/*.csv')

        # Import pkl file
        FallAllD = pd.read_pickle('FallAllD.pkl')

        df = pd.concat([pd.read_csv(f, sep=';') for f in files], ignore_index=True)
        # assign label to df column 'Label'
        df['ActivityID'] = label
        # save columns 'DeviceOrientation', 'AccelerationX', 'AccelerationY', 'AccelerationZ', 'Label' to a new csv file
        df[['DeviceOrientation', 'AccelerationX', 'AccelerationY', 'AccelerationZ', 'Label']].to_csv('./normalized/' + label + '.csv', index=False)
        
        
    df = pd.concat([pd.read_csv(f) for f in glob.glob('./normalized/*.csv')], ignore_index = True)
    return df

def data_normalize(df):
    # for each column in 'Accerlation', 'AccelerationY', 'AccelerationZ',
    # add a new column that is the fft of the original column
    for c in ['AccelerationX', 'AccelerationY', 'AccelerationZ']:
        df[c + '_fft'] = np.fft.fft(df[c])

    # debug purposes
    for c in df.columns:
        print(c)
        
    return df

def data_categorical(df):
    # convert categorical data to numerical data
    df['Label'] = df['Label'].astype('category')
    df['Label'] = df['Label'].cat.codes
    print(df['Label'].unique())

    # convert column `DeviceOrientation` to numerical data
    df['DeviceOrientation'] = df['DeviceOrientation'].astype('category')
    df['DeviceOrientation'] = df['DeviceOrientation'].cat.codes
    return df

def data_split(df):
    x = np.array(df.drop(['Label'], 1)) #drops row 'Label'
    y = np.array(df["Label"])

    #Spliting Data
    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x,y,test_size = 0.3)

    # reshape training and testing data
    y_train = np.array(y_train).reshape(-1,1) # (-1,1) because our data has a single feature, and a 'n' amount of rows
    y_test = np.array(y_test).reshape(-1,1) # (-1,1) bec  ause our data has a single feature, and a 'n' amount of rows
    x_test = np.array(x_test).reshape(x_test.shape[0], x_test.shape[1], 1)

    return x_train, x_test, y_train, y_test

def model_create(x_train):
  model = Sequential()
  model.add(LSTM(128, input_shape=(x_train.shape[1], 1)))
  model.add(Dropout(0.2))
  model.add(Dense(6, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def train_and_accurary_model(model, x_train, x_test, y_train, y_test):
  #reshape the features for the LSTM layer (I REALLY WANT TO FIX UGHHHG)
  x_train = np.array(x_train).reshape(x_train.shape[0], x_train.shape[1], 1)

  # train the model
  model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

  # evaluate the model
  test_loss, test_Acc = model.evaluate(x_test, y_test)
  print('Test accuracy:', test_Acc)
  model.summary()
  return model

def model_save(model):
  # save the model
  model.save('saved_model/my_model')
  return model

In [4]:
#Debugging function
def data_plot(df):
    #debugging function. does not need to be called, or return anything

    # plot the first 100 column 'AccelerationX' and 'AccelerationX_fft'
    df[['AccelerationX', 'AccelerationY', 'AccelerationZ']].iloc[:100].plot()

    # Versus Head (the same)
    #df[['AccelerationX_fft', 'AccelerationY_fft', 'AccelerationZ_fft']].head(100).plot()
    df[['AccelerationX', 'AccelerationY', 'AccelerationZ']].head(100).plot()

In [ ]:
def get_activity(num = 0):
    # a dictionary that maps the case number (ActivityID) with stirng label
    activity_dict = {
    101: 'Fall F, walking, trip',
    102: 'Fall F, walking, trip, rec.',
    103: 'Fall F, walking, slip',
    104: 'Fall F, walking, slip, rec.',
    105: 'Fall F, walking, slip, rot.',
    106: 'Fall F, walking, slip, rot., rec.',
    107: 'Fall B, walking, slip',
    108: 'Fall B, walking, slip, rec.',
    109: 'Fall B, walking, slip, rot.',
    110: 'Fall B, walking, slip rot., rec.',
    111: 'Fall F, walking, syncope',
    112: 'Fall B, walking, syncope',
    113: 'Fall L, walking, syncope',
    114: 'Fall, syncope, table',
    115: 'Fall F, try sit',
    116: 'Fall F, try sit, rec.',
    117: 'Fall B, try sit',
    118: 'Fall B, try sit, rec.',
    119: 'Fall L, try sit',
    120: 'Fall L, try sit, rec.',
    121: 'Fall F, jog, trip',
    122: 'Fall F, jog, trip, rec.',
    123: 'Fall F, jog, slip',
    124: 'Fall F, jog, slip, rev.',
    125: 'Fall F, jog, slip, rot.',
    126: 'Fall F, jog, slip, rot., rec.',
    127: 'Fall L, bed',
    128: 'Fall L, bed, rec.',
    129: 'Fall F, chair, syncope',
    130: 'Fall B, chair, syncope',
    131: 'Fall L, chair, syncope',
    132: 'Fall F, syncope',
    133: 'Fall B, syncope',
    134: 'Fall L, syncope',
    135: 'Fall, syncope, slide over a wall',
    1: 'Start clap hands',
    2: 'Clap hands',
    3: 'Stop clap hands',
    4: 'Clap hands 1',
    5: 'Start wave hands',
    6: 'wave hands',
    7: 'Stop wave hands',
    8: 'Raising hand up',
    9: 'Moving hand down',
    10: 'Move hand up -> down',
    11: 'Hand shaking',
    12: 'Beating a table',
    13: 'Sitting down',
    14: 'Standing up',
    15: 'Fail to stand up',
    16: 'Lying down',
    17: 'Turning while lying',
    18: 'Rising up',
    19: 'Start walking',
    20: 'Walking slowly',
    21: 'Stop walking',
    22: 'Walking quickly',
    23: 'Stumbling',
    24: 'Jogging slowly',
    25: 'Jogging quickly',
    26: 'Jumping slightly',
    27: 'Jumping strongly',
    28: 'B...'
    }
    return activity_dict[num]